# Importing package

In [165]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
sys.path.append('..')
filedirectory = "//Users//joancaceres//Documents//FERBO//FB_W10_S4//run2//"
import scqubits as sq
from scipy.optimize import differential_evolution
from src.utilities import *
from tqdm.notebook import tqdm
from IPython.utils import io
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading data

In [61]:
voltage_period = 3.033
zero_flux = 1.3668

In [62]:
datasets = {}

In [252]:
datanumbers = [
    424403,
    422958,# Large upper scan around phase = 0
    422231, # Close look at phae = pi
    424530, # Large bottom  scan around phase = pi
]

In [303]:
for number in datanumbers:
    datasets[number] = load_data(filedirectory + "FBW10S4_" +str(number))
    datasets[number]['freq'] = datasets[number]['freq']*1e-9
    datasets[number]['phi'] = (datasets[number]['flux']-zero_flux)/voltage_period

['flux']
['flux']
['flux']
['flux']


In [304]:
fig1, ax1 = plot3Ds(
    data_dicts=datasets,
    x_key='phi',
    y_key='freq',
    z_key='phase',
    flatten_horizontal=1,
    flatten_vertical=1,
    vmin=-25,
    vmax=50,
    cmap='gray'
)

# Fluxonium visualization

In [229]:
fluxonium_params = {
    'EJ':   10.58,
    'EC':   1.79,
    'EL':   0.367,
    'flux': 0
}
fluxonium = sq.Fluxonium(**fluxonium_params,cutoff=40)

In [232]:
flux_array = np.linspace(-0.5,0.5,101)
fluxspec = fluxonium.get_spectrum_vs_paramvals(
    param_name='flux',
    param_vals=flux_array,
    evals_count=6,
    subtract_ground=False
)

Spectral data:   0%|          | 0/101 [00:00<?, ?it/s]

In [233]:
evals_matrix = fluxspec.energy_table
transitions0 = evals_matrix - evals_matrix[:,0].reshape(-1,1)
transitions1 = evals_matrix - evals_matrix[:,1].reshape(-1,1)
transitions2 = evals_matrix - evals_matrix[:,2].reshape(-1,1)

transitions0 = np.delete(transitions0,0,axis=1)
transitions1 = np.delete(transitions1,[0,1],axis=1)
transitions2 = np.delete(transitions2,[0,1,2],axis=1)

In [234]:
fig2,ax2 = plt.subplots(1,1)
ax2.plot(flux_array,transitions0, linestyle='--', color='C0', alpha=0.8)
ax2.plot(flux_array,transitions1, linestyle='--', color='C1', alpha=0.8)
plt.show()

# Fluxonium fitting

In [292]:
datafit_dir = "../data/data_fitting/"
transition01 = pd.read_csv(datafit_dir + 'E01.csv', delimiter=';', decimal=',',header=None, names=['phi','e01'])
transition02 = pd.read_csv(datafit_dir + 'E02.csv', delimiter=';', decimal=',',header=None, names=['phi','e02'])
# t02_2 = pd.read_csv(datafit_dir + 'E02_2.csv', delimiter=';', decimal=',',header=None, names=['phi','e02'])
# transition02 = pd.concat([t02, t02_2], ignore_index=True)
transition03 = pd.read_csv(datafit_dir + 'E03_1.csv', delimiter=';', decimal=',',header=None, names=['phi','e03'])

In [296]:
def minimizer(params):
    EJ,EC,EL = params
    EJ,EC,EL = EJ*10,EC,EL/10
    fluxonium = sq.Fluxonium(EJ=EJ,EC=EC,EL=EL,flux=0,cutoff=40)
    
    flux_array1 = transition01['phi'].values
    with io.capture_output() as captured:
        val = fluxonium.get_spectrum_vs_paramvals(param_name='flux', param_vals=flux_array1, evals_count=2, subtract_ground=True)
    e01 = val.energy_table[:,1]
    e01_exp = transition01['e01'].values
    
    flux_array2 = transition02['phi'].values
    with io.capture_output() as captured:
        val = fluxonium.get_spectrum_vs_paramvals(param_name='flux', param_vals=flux_array2, evals_count=3, subtract_ground=True)
    e02 = val.energy_table[:,2]
    e02_exp = transition02['e02'].values
    
    cost_function = np.sqrt(np.sum((e01-e01_exp)**2) + np.sum((e02-e02_exp)**2))/(len(flux_array1) + len(flux_array2))
    
    return cost_function   

In [297]:
bounds_list = [
    (0.8,1.5),
    (1.2,2),
    (2,5)
]

progress = []

# Crear la barra de progreso
progress_bar = tqdm(total=100, desc="Progress")

# Definir la función de callback
def callback(params, convergence):
    # Registrar el estado actual del ajuste
    progress.append(params)
    # Actualizar la barra de progreso
    progress_bar.update(1)
    if len(progress) >= 100:
        progress_bar.close()


result = differential_evolution(func=minimizer,bounds=bounds_list,callback=callback, disp=False, maxiter=100)

Progreso del ajuste:   0%|          | 0/100 [00:00<?, ?it/s]

In [298]:
result

 message: Optimization terminated successfully.
 success: True
     fun: 4.6825108246867605e-05
       x: [ 1.103e+00  1.859e+00  3.663e+00]
     nit: 44
    nfev: 2045
     jac: [-3.990e-07  8.743e-06  2.326e-07]

In [305]:
EJ = result.x[0]*10
EC = result.x[1]
EL = result.x[2]/10

fluxonium_fit = sq.Fluxonium(EJ,EC,EL,flux=0,cutoff=40)
flux_array = np.linspace(-0.7,0.1,4001)
fluxspec = fluxonium_fit.get_spectrum_vs_paramvals(
    param_name='flux',
    param_vals=flux_array,
    evals_count=6,
    subtract_ground=False
)
evals_matrix = fluxspec.energy_table
transitions0 = evals_matrix - evals_matrix[:,0].reshape(-1,1)
transitions1 = evals_matrix - evals_matrix[:,1].reshape(-1,1)
transitions2 = evals_matrix - evals_matrix[:,2].reshape(-1,1)

transitions0 = np.delete(transitions0,0,axis=1)
transitions1 = np.delete(transitions1,[0,1],axis=1)
transitions2 = np.delete(transitions2,[0,1,2],axis=1)

Spectral data:   0%|          | 0/4001 [00:00<?, ?it/s]

In [306]:
fig1.suptitle(f'EJ = {np.round(EJ,2)}, EC = {np.round(EC,2)}, EL = {np.round(EL,3)}')
alpha = 0.5
plot_tran0 = ax1.plot(flux_array,transitions0, linestyle='--', linewidth=0.8, color='C0', alpha=alpha)
plot_tran1 = ax1.plot(flux_array,transitions1, linestyle='--', linewidth=0.8, color='C1', alpha=alpha)
fig1.tight_layout()
plt.show()

In [300]:
for line in plot_tran0:
    line.remove()
for line in plot_tran1:
    line.remove()